In [17]:
!python train_skeleton.py \
  --skeleton_dir /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton \
  --window 90 --stride 30 \
  --batch_size 32 --epochs 30

/home/qsw26/smartfall/gait_loss/latent_diff_skeleton/train_skeleton.py:24: UserWarning: genfromtxt: Empty input file: "/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T01.csv"
  arr = np.genfromtxt(
[SKIP] /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T01.csv: empty array
/home/qsw26/smartfall/gait_loss/latent_diff_skeleton/train_skeleton.py:24: UserWarning: genfromtxt: Empty input file: "/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T02.csv"
  arr = np.genfromtxt(
[SKIP] /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T02.csv: empty array
/home/qsw26/smartfall/gait_loss/latent_diff_skeleton/train_skeleton.py:24: UserWarning: genfromtxt: Empty input file: "/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T03.csv"
  arr = np.genfromtxt(
[SKIP] /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T03.csv: empty array
/home/qsw26/smartfall/gai

In [20]:
!python train_sensor.py \
  --sensor_dir1 /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_wrist \
  --sensor_dir2 /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_hip \
  --num_classes 12 \
  --epochs 30 \
  --batch_size 64 \
  --window 90 \
  --stride 30 \
  --use_last_3_cols

[Dataset] common_files=1691 valid_files=1682 skipped=9 total_windows=18410
Epoch 1/30 [val]: 100%|████████████████████████| 58/58 [00:00<00:00, 297.86it/s]
Epoch 1: train_loss=2.1755 val_loss=2.0478 val_acc=29.85%
Saved: best_sensor_model.pth
Epoch 2/30 [val]: 100%|████████████████████████| 58/58 [00:00<00:00, 317.00it/s]
Epoch 2: train_loss=2.0365 val_loss=1.9859 val_acc=28.60%
Saved: best_sensor_model.pth
Epoch 3/30 [val]: 100%|████████████████████████| 58/58 [00:00<00:00, 323.35it/s]
Epoch 3: train_loss=1.9246 val_loss=1.9421 val_acc=28.84%
Saved: best_sensor_model.pth
Epoch 4/30 [val]: 100%|████████████████████████| 58/58 [00:00<00:00, 379.60it/s]
Epoch 4: train_loss=1.8214 val_loss=1.8381 val_acc=34.79%
Saved: best_sensor_model.pth
Epoch 5/30 [val]: 100%|████████████████████████| 58/58 [00:00<00:00, 369.39it/s]
Epoch 5: train_loss=1.8098 val_loss=1.7040 val_acc=38.00%
Saved: best_sensor_model.pth
Epoch 6/30 [val]: 100%|████████████████████████| 58/58 [00:00<00:00, 375.48it/s]
Epoc

In [23]:
!python train_diffusion.py \
  --skeleton_dir /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton \
  --sensor_dir1 /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_wrist \
  --sensor_dir2 /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_hip \
  --num_classes 12 \
  --window 90 --stride 30 \
  --timesteps 1000 \
  --epochs 50 \
  --batch_size 64 \
  --use_last_3_cols \
  --out best_diffusion_model.pth

/home/qsw26/smartfall/gait_loss/latent_diff_skeleton/train_diffusion.py:18: UserWarning: genfromtxt: Empty input file: "/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T01.csv"
  arr = np.genfromtxt(
[SKIP] /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T01.csv: empty/bad read
/home/qsw26/smartfall/gait_loss/latent_diff_skeleton/train_diffusion.py:18: UserWarning: genfromtxt: Empty input file: "/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T02.csv"
  arr = np.genfromtxt(
[SKIP] /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T02.csv: empty/bad read
/home/qsw26/smartfall/gait_loss/latent_diff_skeleton/train_diffusion.py:18: UserWarning: genfromtxt: Empty input file: "/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T03.csv"
  arr = np.genfromtxt(
[SKIP] /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton/S50A01T03.csv: empty/bad read
/home/qsw26/s

In [42]:
!python generate.py \
  --imu_dir1 //home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_wrist\
  --imu_dir2 /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_hip\
  --stage1_ckpt checkpoints/smoke_stage1.pt \
  --stage2_ckpt checkpoints/smoke_stage2_accel_only.pt \
  --stage3_ckpt checkpoints/smoke_stage3_accel_only.pt \
  --max_files 30 \
  --num_gifs 10

Loading IMU CSVs:  50%|████████████            | 15/30 [00:00<00:00, 143.50it/s][skip] S30A02T03.csv: too short T=86 < window=90
[skip] S30A02T05.csv: too short T=87 < window=90
Generating+Decoding: 100%|████████████████████| 144/144 [14:37<00:00,  6.10s/it]

Saved decoded windows (x_hat) to: outputs/generated_decoded.pt
Saved 10 GIFs to: outputs/gifs


In [31]:
import torch

obj = torch.load("outputs/generated.pt", map_location="cpu")
print(type(obj))

# if dict
if isinstance(obj, dict):
    for k,v in obj.items():
        if hasattr(v, "shape"):
            print(k, v.shape, v.dtype, v.numel()*v.element_size()/1e9, "GB")
        else:
            print(k, type(v))

# if list[dict]
elif isinstance(obj, list) and len(obj) and isinstance(obj[0], dict):
    keys = obj[0].keys()
    print("keys:", keys)
    for k in keys:
        v = obj[0][k]
        if hasattr(v, "shape"):
            print("example", k, v.shape, v.dtype)

/tmp/ipykernel_539866/1079165402.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  obj = torch.load("outputs/generated.pt", map_location="cpu")


<class 'list'>
keys: dict_keys(['file', 'start', 'z0'])
example z0 torch.Size([90, 32, 256]) torch.float32


In [32]:
import torch
import numpy as np

# ===== EDIT THESE =====
PT_IN = "outputs/generated.pt"
PT_OUT = "outputs/generated_decoded.pt"
STAGE1_CKPT = "checkpoints/stage1.pt"   # must contain decoder weights
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ---- import your repo modules (edit paths if needed) ----
from diffusion_model.skeleton_model import SkeletonStage1Model
from diffusion_model.util import build_chain_adjacency

# ===== Load latents =====
items = torch.load(PT_IN, map_location="cpu")  # list of dicts: {file,start,z0}
assert isinstance(items, list)

# Peek one latent to infer dims
z0 = items[0]["z0"]  # [W,32,256]
W, J, D = z0.shape
print("latent shape:", z0.shape)

# ===== Load Stage-1 model (decoder) =====
model = SkeletonStage1Model(
    joint_dim=3,
    latent_dim=D,
    hidden_dim=256,          # must match training
    num_layers=3,            # must match training
    num_heads=8,             # must match training
    diffusion_steps=500,     # not used for decode, but ctor may require
).to(DEVICE)

ckpt = torch.load(STAGE1_CKPT, map_location="cpu")

# Most repos store decoder separately or inside dict
# Try common patterns:
if isinstance(ckpt, dict) and "decoder" in ckpt:
    model.decoder.load_state_dict(ckpt["decoder"], strict=False)
else:
    # If your stage1.pt was saved as decoder state_dict only:
    model.decoder.load_state_dict(ckpt, strict=False)

model.eval()

# adjacency [J,J]
adj = build_chain_adjacency(J, include_self=True, device=torch.device(DEVICE))

# ===== Decode all items (optionally only first N to save time) =====
decoded = []
with torch.no_grad():
    for it in items:
        z = it["z0"].to(DEVICE)           # [W,32,256]
        z = z.unsqueeze(0)                # [1,W,32,256] if decoder expects batch
        x_hat = model.decoder(z, adj)     # expected [1,W,32,3] (depends on your decoder)
        x_hat = x_hat.squeeze(0).cpu()    # [W,32,3]

        out = dict(it)
        out["x_hat"] = x_hat
        decoded.append(out)

torch.save(decoded, PT_OUT)
print("Saved decoded pt:", PT_OUT)
print("Example x_hat shape:", decoded[0]["x_hat"].shape)

/tmp/ipykernel_539866/2415149608.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  items = torch.load(PT_IN, map_location="cpu")  # list of dicts: {file,start,z0}


latent shape: torch.Size([90, 32, 256])


/tmp/ipykernel_539866/2415149608.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(STAGE1_CKPT, map_location="cpu")


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/stage1.pt'

In [34]:
!python pt_to_gif.py --pt_in outputs/generated.pt --out_dir outputs/gifs --max_items 10 --fps 30 --stride 1

usage: pt_to_gif.py [-h] --pt_in PT_IN --decoder_ckpt DECODER_CKPT
                    [--out_dir OUT_DIR] [--max_items MAX_ITEMS]
                    [--device DEVICE] [--num_joints NUM_JOINTS]
                    [--latent_dim LATENT_DIM] [--hidden_dim HIDDEN_DIM]
                    [--num_layers NUM_LAYERS] [--num_heads NUM_HEADS]
                    [--fps FPS] [--stride STRIDE]
pt_to_gif.py: error: the following arguments are required: --decoder_ckpt
